In [1]:
!nvidia-smi

Sat Nov  4 04:18:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

data = load_dataset("re2panda/click_bate_1000_final")

data['train']

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/11400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/600 [00:00<?, ? examples/s]

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 11400
})

In [ ]:
data['train'][0]

{'output': '해당기사는 정상기사입니다. 낚시기사 유형은 0 입니다.',
 'input': '기사제목: f\\"과징금 부과만 몰두\\" 지적에…금융위, 수납률 확 늘렸네 기사내용: f금융당국이 금융사에 부과한 과징금의 수납액을 지난해 대폭 늘린 것으로 파악됐다.\n과징금 부과에만 몰두하고 거둬들이려는 노력이 부족하다는 비판에 대안을 마련한 효과다.\n6일 금융당국에 따르면 지난해 금융위원회가 부과한 과징금 징수결정액은 512억5900만원으로 집계됐다.\n이중 실제로 거둬들인 수납액은 337억9400만원이다.\n수납률은 65.9%로 미수납액은 174억6500만원이었다.\n금융위는 법률을 위반한 금융기관이나 종사자에 금전적 제재수단으로 과징금을 부여한다.\n법을 어겨 얻은 경제적 이익을 박탈하거나 의무를 충실하게 이행하도록 유도하기 위해서다. 2020년만 해도 금융위의 과징금 수납률은 52.5%로 절반 수준이었다.\n440억6300만원을 납부받아야 하는데 거둔 돈이 231억8600만원에 불과했다.\n이 중 금융투자업권이 54%(151억9800만원)로 가장 높았고 보험업 46%(18억3400만원), 저축은행 35%(37억1300만원) 순이었다.\n2019년에도 396억6300만원의 과징금 결정액 중에서 176억700만원만 거둬 수납률은 44.4%에 그쳤다.\n금융당국은 그동안 수납률이 떨어지는 상황에 대해 인정하면서도 인력부족과 대상업체의 폐업 및 재력부족 등을 이유로 수납의 어려움을 토로했었다.\n이에 대해 국회는 2008년부터 회계연도 결산심사를 진행하면서 금융위가 징수율 제고를 위한 노력을 해야 한다고 주문했다.\n문제가 해결되지 않자 2020년 결산심사 때에도 과징금 부과 시 걸리는 기간을 단축하고, 은닉자산을 추적하기 위한 실효성 있는 징수방안을 강구하라고 요구했다.\n그럼에도 시정조치 요구가 이어지자 금융위는 연 12회로 진행하던 현장 방문재산 조사를 분기별 1회로 늘리겠다고 국회에 보고했다.\n또 체납액 분할 납부 유도를 통해 수납률을 높이겠다고

In [ ]:
data['test'][0]

{'output': '해당기사는 낚시성기사입니다. 낚시기사 유형은 99 입니다.',
 'input': '기사제목: f경기도, 방치폐기물 우려사업장 특별점검 87개소 적발 기사내용: f경기도는 지난해 12월 17일부터 지난달 11일까지 도내 방치폐기물 우려사업장을 대상으로 특별점검을 실시한 결과, 위반업체 87개소를 적발했다고 5일 밝혔다.\n지난해 12월 수립한 불법방치 폐기물 근절 종합대책에 따라 도내 방치폐기물 업체에 대한 관리 감독을 강화함으로써 폐기물로부터 안전한 경기도를 조성하고자 마련된 이번 점검은 폐기물을 방치할 우려가 높은 사업장 583개소를 대상으로 실시됐다.\n도는 도내 31개 시군과 합동으로 실시한 이번 점검에서 폐기물을 방치하다 1회 이상 적발된 이력이 있는 사업장 및 경영부실 업체 등을 중심으로 보관기준 위반 처리기준 위반 보증보험 미갱신 허가신고 미이행 여부 등을 집중 단속했다.\n점검 결과, 보관기준 위반이 24건으로 가장 많았고, 이어 허가신고(변경) 미이행 13건, 불법소각 등 처리기준 위반 8건, 보증보험 미갱신 5건, 올바로시스템 미입력 등 기타 37건 등 총 87개 업체가 적발됐다.\n이에 도는 위반업체를 관할 시군에 통보, 영업정지 및 과태료 부과 등 행정처분이 진행되도록 하는 조치하는 한편 중대한 위반 행위를 저지른 업체 24개소에 대해서는 형사 고발을 통해 행위자 및 관련자들에 대한 엄중 처벌이 이뤄지도록 했다.\n이와 더불어 도는 필리핀 불법수출 폐기물로 인해 발생하고 있는 사회적, 환경적 문제를 해결하기 위해 오는 6일부터 오는 29일까지 수출신고를 취득한 도내 폐기물처리업체 38곳을 대상으로 특별단속을 실시하는 등 폐기물에 대한 관리 감독을 강화해 나갈 방침이다.\n김건 경기도 환경국장은 도내 불법투기방치폐기물을 안전하고 신속하게 처리하고 폐기물 배출처리에 대한 지도점검을 강화해 불법 폐기물 발생을 근절하고 예방해 나갈 계획이라며 도민이 깨끗하고 쾌적한 환경에서 살 수 있도록 최선의 노력을 다하겠다고 말했다.',


In [ ]:
#load a model

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-12.8b"
# 4비트 양자화를 활성화하는 BitsAndBytes 구성을 만듭니다.
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,                       # 모델을 4비트로 로딩하지 않습니다.
    bnb_8bit_use_double_quant=True,          # 8비트 양자화에 이중 양자화를 사용합니다.
    bnb_8bit_quant_type="nf8",               # "nf8" 양자화 유형을 사용합니다 (노이즈 프리 8비트).
    bnb_8bit_compute_dtype=torch.bfloat16    # 계산 dtype으로 bfloat16을 사용합니다.
)


tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:
from peft import prepare_model_for_kbit_training

# gradient_checkpointing을 활성화합니다.
model.gradient_checkpointing_enable()

# 모델을 8비트로 변환합니다.
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0

    # 모델의 모든 매개변수를 반복합니다.
    for _, param in model.named_parameters():

        # 모든 매개변수의 크기를 더합니다.
        all_param += param.numel()

        # 훈련 가능한 매개변수의 크기를 더합니다.
        if param.requires_grad:
            trainable_params += param.numel()

    # 훈련 가능한 매개변수 수, 전체 매개변수 수, 훈련 가능한 매개변수 비율을 출력합니다.
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

# LoRA 구성 객체를 생성합니다.
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 모델을 LoRA로 변환합니다.
model = get_peft_model(model, config)

# 모델의 훈련 가능한 매개변수 수를 출력합니다.
print_trainable_parameters(model)

trainable params: 6553600 || all params: 12900157440 || trainable%: 0.05080248074863806


--> 학습가능한 매개변수 비율 약 12%

In [ ]:
# 모델 max_length 확인
from transformers import AutoModelForCausalLM

# 모델을 로드합니다.
load_model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-1.3b")

# 최대 시퀀스 길이를 확인합니다.
max_length = load_model.config.max_position_embeddings
print(f"The maximum length for the model is: {max_length}")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The maximum length for the model is: 2048


In [ ]:
CUTOFF_LEN = 256

In [ ]:
# `generate_prompt` 함수는 주어진 데이터 포인트를 사용하여 인스트럭션, 입력, 응답을 포함하는 프롬프트를 생성합니다.
def generate_prompt(data_point):
    # f-string을 사용하여 인스트럭션, 입력, 응답을 포맷팅된 문자열로 구성합니다.
    return f"""아래는 컨텍스트를 제공하는 입력과 짝을 이루는 작업을 설명하는 지침입니다. 요청을 적절하게 완료하는 응답을 작성합니다.
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""

# `tokenize` 함수는 주어진 프롬프트를 토큰화합니다.
def tokenize(prompt, add_eos_token=True):
    # 토크나이저를 사용하여 프롬프트를 토큰화합니다. 여기서 CUTOFF_LEN은 최대 토큰 길이를 정의합니다.
    result = tokenizer(
        prompt,
        truncation=True,          # 너무 긴 입력은 자르기
        max_length=CUTOFF_LEN,    # 최대 길이 설정
        padding=False,            # 패딩을 추가하지 않음
        return_tensors=None,      # 반환 텐서 유형을 지정하지 않음
    )
    # 마지막 토큰이 EOS 토큰이 아니고, 길이가 CUTOFF_LEN 미만이며, EOS 토큰을 추가해야 하는 경우
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)  # EOS 토큰을 추가합니다.
        result["attention_mask"].append(1)  # 어텐션 마스크에 1을 추가합니다.

    # 레이블을 입력 ID와 동일하게 설정합니다.
    result["labels"] = result["input_ids"].copy()

    return result  # 결과를 반환합니다.

# `generate_and_tokenize_prompt` 함수는 데이터 포인트를 받아 프롬프트를 생성하고 토큰화합니다.
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)  # 프롬프트를 생성합니다.
    tokenized_full_prompt = tokenize(full_prompt)  # 프롬프트를 토큰화합니다.
    return tokenized_full_prompt  # 토큰화된 프롬프트를 반환합니다.


In [ ]:
train_data = (data["train"].map(generate_and_tokenize_prompt))

val_data = (data["test"]).map(generate_and_tokenize_prompt)

Map:   0%|          | 0/11400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
# 필요한 라이브러리 import
import transformers
from transformers import TrainingArguments, Trainer

# # 사용할 토크나이저 설정
# tokenizer.pad_token = tokenizer.eos_token

# # AdamW optimizer 활성화
# optimizer = transformers.AdamW(model.parameters())

# scheduler 설정
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=training_args.warmup_steps,
    num_training_steps=training_args.max_steps,
)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir="outputs",  # 출력 디렉토리 설정
    per_device_train_batch_size=5,  # 학습 시 기기당 배치 크기 설정
    per_device_eval_batch_size=5,  # 평가 시 기기당 배치 크기 설정
    gradient_accumulation_steps=2,  # 그래디언트 업데이트를 수행하기 전에 축적할 업데이트 단계 수 설정
    max_steps=-1,  # 수행할 총 학습 단계 수 설정
    learning_rate=5e-3,  # 시작 학습률 설정
    fp16=True,  # 혼합 정밀도 학습 사용 (Nvidia Apex 또는 AMP 라이브러리 필요)
    logging_steps=10,  # 10단계마다 학습 정보 로깅
    optim="adamw_torch",  # 최적화기 (torch에서 제공하는 최적화기 사용, 더 안정적일 수 있음)
    report_to="tensorboard",  # TensorBoard에 로깅 활성화
    lr_scheduler_type="linear",  # 학습률 스케줄러 유형 (선형 스케줄러)
    warmup_steps=50,  # 학습률 스케줄러의 예열 단계 수 설정
    weight_decay=0.01,  # 정규화를 위한 가중치 감소 설정
    max_grad_norm=1.0,  # 그래디언트의 최대 노름 설정 (그래디언트 클리핑)
    evaluation_strategy="steps",  # 단계마다 평가
    eval_steps=100,
    load_best_model_at_end=True,  # 학습이 끝나면 최고의 모델 로드
    metric_for_best_model="eval_loss",  # 최고의 모델을 위한 평가 지표
)

# 모델, 학습 인자, 학습 데이터셋, 데이터 콜레이터를 사용하여 Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),  # 데이터 처리를 위한 콜레이터
)

# 학습 프로세스 시작
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...

In [ ]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 195362
Number of validation examples: 21707


In [ ]:
# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/195362 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/21707 [00:00<?, ? examples/s]

In [ ]:
data['train']['text']

['### 질문: 기사제목: f노무현 자서전 정리자 유시민, 유OO된 까닭 기사내용: f노무현 전 대통령의 서거 1주년을 맞아 출간된 자서전 광고에 대해 중앙선거관리위원회가 제동을 걸고 나섰다.\n선관위는 최근 광고에 이 책의 정리자로 유시민 경기도지사 후보의 이름이 들어가 있는 것이 선거법을 위반했다며 시정을 요구했다.\n출판사 돌베게 쪽은 선관위에 책 광고에서 그 책을 정리해 펴낸 사람의 이름을 밝히는 것은 빠뜨릴 수 없는 서지정보이며, 개인을 부각하거나 홍보하는 문구도 전혀 없다고 설명했으나 선관위는 이를 받아들이지 않았다.\n이에 따라 출판사는 13일 이후부터 신문에 게재되는 광고에서는 정리자의 이름을 \'유○○\'으로 수정해 싣기로 결정했다.\n출판사 관계자는 \\"서지정보가 빠지거나 불분명한 상태로 책 광고를 하는 것은 상식적으로 납득하기 어려운 일이지만, 노무현 전 대통령의 1주기를 앞둔 시점에 출판사가 해당 책의 광고를 중지할 수도 없는 상황\\"이라며 \\"부득이하게 정리자의 명의를 \\"유○○\\"로 표시하기로 한 것\\"이라고 설명했다.\n선관위는 이에 대해 14일 \\"선거법 93조에 정당의 명칭 또는 후보자의 성명을 나타내는 사진이나 인쇄물 등을 게시할 수 없도록 한 것을 위반했다\\"고 말했다.\n그러나 일각에서는 선관위가 단순 서지정보까지 제동을 걸고 나선 것은 유 후보의 후보단일화 성공과 노 대통령 추모 1주기 등 정치적 사안을 고려한 것 아니겠느냐는 관측이 나오고 있다.\n유 후보는 지난 2009년 노 전 대통령의 장례 직후 자서전 집필을 의뢰받고 6개월 이상 정리작업에 매진했으며, 현재 는 출간 2주 만에 온라인, 오프라인 서점 집계에서 8만 부 이상 판매되며 베스트셀러에 올랐다.\n\n### 답변: 해당 기사는 정상기사입니다.<|endoftext|>',
 '### 질문: 기사제목: f\'中 침공 대비 훈련\' 대만 \\"우크라 전쟁 교훈 삼겠다\\" 기사내용: f대만이 중국군 공격에 대비하는 올해 주요 군사훈련을 준비하는 과정

In [ ]:
%load_ext tensorboard
%tensorboard --logdir outputs/runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 9223), started 0:00:18 ago. (Use '!kill 9223' to kill it.)

<IPython.core.display.Javascript object>

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.generate(**tokenizer("### 질문: Martinez 씨는 그의 딸인 Chiquita보다 2피트 더 큽니다. Chiquita의 키가 5피트이면 합친 키를 계산하십시오.", return_tensors='pt', return_token_type_ids=False))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1264: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1272: UserWarning: Input length of input_ids is 46, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1580: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by cal

torch.Size([1, 46])
no pkv
torch.Size([1, 46])


tensor([[    6,     6,     6,  2438,    29,  1663, 10301, 12509,    93,  1235,
           272,   353,   285,  2586,   333, 16860,    76, 19928,  3883,    68,
           886,   401, 19457,   715,   744,   125,   623,    17, 16860,    76,
         19928,  3883,    68,   285,  1639,   293,   648, 19457, 26480, 16777,
          1639,   301,  4954,   284,  5451,    17,    15]])

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [ ]:
gen("기사제목: f풀무원녹즙, 1등급 원유 담은 '프로틴 음료' 출시 기사내용: f풀무원녹즙(대표 김기석)은 매일 신선하게 단백질과 칼슘 섭취를 돕는 프로틴 음료 '핸디밀 프로틴 PRO'를 출시했다고 23일 밝혔다. '핸디밀 프로틴 PRO'는 아침 식사 대용은 물론 운동 전후 즐기기 좋은 프로틴 제품이다. 150ml 제품 한 병으로 매일 125ml 우유 2팩과 비슷한 양의 단백질을 섭취할 수 있다. 풀무원녹즙 관계자는 미디어SR에 \"핸디밀 프로틴 PRO'는 국내산 1등급 원유를 사용해 근력을 보충해 줄 단백질을 매일 신선하게 보충할 수 있다\"고 설명했다. 여기에 콩에서 유래한 식물성 단백질을 최적의 비율로 더해 건강함을 높였다. 검은약콩, 참깨 등이 고소함과 단백함을 더해준다. 신선한 단백질에 프리바이오틱스도 함께 담아 더욱 건강하게 즐길 수 있다. '핸디밀 프로틴 PRO'하루 한 병으로 매일 충분한 칼슘 섭취도 가능하다. 한 병에 칼슘 220mg 담아 하루 한 병 섭취로 190ml 우유 한 팩에 들어있는 칼슘보다 더 많은 칼슘을 섭취할 수 있다. 또한, 한 병에 칼로리가 150kcal, 제품에 함유된 탄수화물 함량이 흰쌀밥 한 공기(190g)의 1/4 정도에 불과해 부담 없는 아침 식사로 제격이다. 풀무원녹즙 이찬미 PM(제품 매니저)은 \"코로나19 영향으로 건강에 대한 관심이 커지면서 간편한 한 끼 대용식은 물론 운동 전후 맛있고 신선하게 즐길 수 있는 프로틴 음료에 대한 관심이 높아지고 있다\"며 \"휴대하기 편한 150ml 핸디사이즈에, 근육을 형성하는 단백질까지 충분히 챙길 수 있는'핸디밀 프로틴 PRO'로 매일 아침 간편하게 건강을 챙길 수 있다\"고 말했다.")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<s> ### 질문: 기사제목: f풀무원녹즙, 1등급 원유 담은 '프로틴 음료' 출시 기사내용: f풀무원녹즙(대표 김기석)은 매일 신선하게 단백질과 칼슘 섭취를 돕는 프로틴 음료 '핸디밀 프로틴 PRO'를 출시했다고 23일 밝혔다. '핸디밀 프로틴 PRO'는 아침 식사 대용은 물론 운동 전후 즐기기 좋은 프로틴 제품이다. 150ml 제품 한 병으로 매일 125ml 우유 2팩과 비슷한 양의 단백질을 섭취할 수 있다. 풀무원녹즙 관계자는 미디어SR에 "핸디밀 프로틴 PRO'는 국내산 1등급 원유를 사용해 근력을 보충해 줄 단백질을 매일 신선하게 보충할 수 있다"고 설명했다. 여기에 콩에서 유래한 식물성 단백질을 최적의 비율로 더해 건강함을 높였다. 검은약콩, 참깨 등이 고소함과 단백함을 더해준다. 신선한 단백질에 프리바이오틱스도 함께 담아 더욱 건강하게 즐길 수 있다. '핸디밀 프로틴 PRO'하루 한 병으로 매일 충분한 칼슘 섭취도 가능하다. 한 병에 칼슘 220mg 담아 하루 한 병 섭취로 190ml 우유 한 팩에 들어있는 칼슘보다 더 많은 칼슘을 섭취할 수 있다. 또한, 한 병에 칼로리가 150kcal, 제품에 함유된 탄수화물 함량이 흰쌀밥 한 공기(190g)의 1/4 정도에 불과해 부담 없는 아침 식사로 제격이다. 풀무원녹즙 이찬미 PM(제품 매니저)은 "코로나19 영향으로 건강에 대한 관심이 커지면서 간편한 한 끼 대용식은 물론 운동 전후 맛있고 신선하게 즐길 수 있는 프로틴 음료에 대한 관심이 높아지고 있다"며 "휴대하기 편한 150ml 핸디사이즈에, 근육을 형성하는 단백질까지 충분히 챙길 수 있는'핸디밀 프로틴 PRO'로 매일 아침 간편하게 건강을 챙길 수 있다"고 말했다.

### 답변:

* 내용 읽기	3차 기간
* 개요
	1. 질문: 기사제목: f풀무원녹즙, 1등급 원유 담은 '프로틴 음료' 출시   에 대한 풀무원녹즙의 입장은 ..... 건강에 관중 있다고 정말 오늘 소식을 전한다.
	2. 최고 의도: 1등급 원유를 사용한 임프로틴프로틴, 오늘 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

model.push_to_hub("re2panda/polyglot_12b_grade_school_math3", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:844: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/re2panda/polyglot_12b_grade_school_math2/commit/36fd1cd6b14e7bbf25840e83b81c5dad1d120652', commit_message='Upload model', commit_description='', oid='36fd1cd6b14e7bbf25840e83b81c5dad1d120652', pr_url=None, pr_revision=None, pr_num=None)

## Model launch from hugging face

# 새 섹션

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

model_id = "EleutherAI/polyglot-ko-12.8b"
peft_model_id = "re2panda/polyglot_12b_grade_school_math2"

config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=False,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.eval()

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=5120, out_features=15360, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [ ]:
gen("각 상자에는 180개의 사과가 있었습니다. 12개의 그러한 사과 상자가 공장으로 배달되었습니다. 160개의 사과가 썩어서 버려야 했습니다. 나머지 사과는 각각 20개의 사과 상자에 포장되었습니다. 사과가 몇 상자 있었나요?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 각 상자에는 180개의 사과가 있었습니다. 12개의 그러한 사과 상자가 공장으로 배달되었습니다. 160개의 사과가 썩어서 버려야 했습니다. 나머지 사과는 각각 20개의 사과 상자에 포장되었습니다. 사과가 몇 상자 있었나요?

### 답변: 12개의 상자 중 12/10 = 사과 하나가 손상됐습니다.
따라서 그들은 160-1=159개의 사과를 남겼습니다.
이것은 159*20 = 2895개의 사과를 남겼습니다.
상자에는 각 상자에 180이 있었으므로 2895-180 = 2170개의 사과 상자가 있었습니다.
 답은 2170입니다.
,
### 답변: 상자당 사과 1520개가 있었습니다.
따라서 80개의 상자에 20*80 = 1680개의 사과가 있었습니다.
한 상자에 180개의 사과가 있었고 모두 1520개의 상자가 있었다는 것을 의미하므로 사과에서 180*1520= 2880개의 사과가 나왔음을 의미합니다.
따라서 사과로부터 20*80= 1600개의 사과가 나왔다는 것을 의미합니다.
따라서 사과 상자 1680개 + 마일리지 1600개 + 1520개 = 5486개의 사과가 있었습니다.
 답은 5486입니다.
 답은 5486입니다.
 답은 1876이고 답은 1876입니다.
 답은 5486입니다.


In [ ]:
model_id = "EleutherAI/polyglot-ko-12.8b"
peft_model_id = "re2panda/polyglot_12b_grade_school_math2"

config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=False,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.eval()

NameError: ignored